In [5]:
#%pip install pyaudio -q
%pip install --upgrade pip -q
%pip install python-dotenv langchain_openai langchain_community chromadb youtube-transcript-api pytube pypdf web3 -q
%pip install youtube-transcript-api bs4 pypdf -q
%pip install SpeechRecognition -q 
%pip install opencv-python -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import WebBaseLoader, YoutubeLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pytube import YouTube
from web3 import Web3
import requests

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
############################# Document instances
## Create a crawl to list all the webpages, and then load them here
web1 = WebBaseLoader("https://www.fidesinnova.io/")
web2 = WebBaseLoader("https://fidesinnova.io/devices/")
web3 = WebBaseLoader("https://fidesinnova.io/Contacts/")
web4 = WebBaseLoader("https://fidesinnova.io/courses/")
web5 = WebBaseLoader("https://linktr.ee/fidesinnova/")
web6 = WebBaseLoader("https://explorer.fidesinnova.io/")
web7 = WebBaseLoader("https://play.google.com/store/apps/details?id=io.fidesinnova.front&pli=1/")
web8 = WebBaseLoader("https://apps.apple.com/ca/app/fidesinnova/id6477492757/")
web9 = WebBaseLoader("https://fidesinnova.io/blog-standard/")                     
web10 = WebBaseLoader("https://fidesinnova.io/About/")
web11= WebBaseLoader("https://fidesinnova.io/#articles/")                    
web12= WebBaseLoader("https://discord.com/invite/NQdM6JGwcs/")
web13= WebBaseLoader("https://www.youtube.com/@FidesInnova/playlists/")
web14= WebBaseLoader("https://x.com/FidesInnova/")
web15= WebBaseLoader("https://fidesinnova.io/service-market/")
web16= WebBaseLoader("https://fidesinnova.io/service-contract-2/")
web17= WebBaseLoader("https://fidesinnova.io/d2pos/")
web18= WebBaseLoader("https://fidesinnova.io/web3/")
web19= WebBaseLoader("https://fidesinnova.io/service-contract/")
web20= WebBaseLoader("https://fidesinnova.io/miotn/")
web21= WebBaseLoader("https://fidesinnova.io/consensus-algorithms/")
web22= WebBaseLoader("https://fidesinnova.io/mqtt-mqtts/")
                                                                                   

## Adding all repositories
git1 = WebBaseLoader("https://github.com/TheArchitect2000/iot-server/")
git2 = WebBaseLoader("https://github.com/TheArchitect2000/zkiot-riscv-qemu-c/")
git3 = WebBaseLoader("https://github.com/TheArchitect2000/Fides-Innova-WiKi/")
git4 = WebBaseLoader("https://github.com/TheArchitect2000/Fides-Innova-WiKi/blob/main/fides-zero-knowledge-proof-zkp-algorithm/1-setup-phase/1-1-setup.md/")
git5 = WebBaseLoader("https://github.com/TheArchitect2000/zkiot-arm-siemens-iot2050-c.git/")


## Adding all PDfs
pdf1 = PyPDFLoader("zkIoT.pdf")

## Adding all YouTube videos
video1 = YoutubeLoader("kgYxyxeDNl4")
video2 = YoutubeLoader("VZMBE2NLSC4")
video3 = YoutubeLoader("YpfFHI3Ivmo")
video4 = YoutubeLoader("Rfl3rjQor-o")
video5 = YoutubeLoader("FtELZXp3qmQ")
video6 = YoutubeLoader("loHb12HmJas")
video7 = YoutubeLoader("sPWmDRItCNk")
video8 = YoutubeLoader("1DAkWOP_uY4")
video9 = YoutubeLoader("2R_bDZ0sasM")
video10 = YoutubeLoader("3GVfyu4uzhs")
video11 = YoutubeLoader("c8dQtxByOcY")
video12 = YoutubeLoader("RcaMNALojIo")
video13 = YoutubeLoader("qMjLWztjqvk")
video14 = YoutubeLoader("DzvQhVw0tNU")
video15 = YoutubeLoader("asw_A0VSObI")                      
video16 = YoutubeLoader("dBEEPLjEJGU")
video17 = YoutubeLoader("s1ZPMrqU0c0")
video18 = YoutubeLoader("K_7RhSkWgbM")
video19 = YoutubeLoader("92QUILNAhDI")
video20 = YoutubeLoader("WZnxV-f6E7k")
video21 = YoutubeLoader("tNWEXETid7Q")
video22 = YoutubeLoader("8Jk19GqcEW4")                     
video23 = YoutubeLoader("Rfl3rjQor-o")
video24 = YoutubeLoader("zPfHYADmUGY")
video25 = YoutubeLoader("2GC05MOMqKY")
video26 = YoutubeLoader("kqMhYgLBe1o")                      
video27 = YoutubeLoader("Mb0KHJFqg5Q")
video28 = YoutubeLoader("A3oSsCWd7YM")
video29 = YoutubeLoader("LlnLmo1GabE")
video30 = YoutubeLoader("xepw8B3ziIU")                                             

## Fides Blockchain
rpc_url = "https://rpc1.fidesinnova.io"
w3 = Web3(Web3.HTTPProvider(rpc_url))

if w3.is_connected():
    print("✅ Connected to FidesInnova Blockchain!")
    print("Latest block:", w3.eth.block_number)
else:
    print("❌ Connection failed.")

contract_address = Web3.to_checksum_address("0x4b08ea934e6bfb7c72a376c842c911e1dd2aa74f")
with open("NodeServiceDeviceManagement.abi") as f1:
   abi =  f1.read() # Smart contract ABI

contract = w3.eth.contract(address=contract_address, abi=abi)

#sensor_data = contract.functions.getSensorData(sensor_id).call()

############################# Loading
loaders = [web1, web2, web3, web4, web5, web6, web7, web8, web9, web10, web11, web12, web13, web14, web15, web16, web17, web18, web19, web20, web21, web22,  git1, git2, git3, git4, git5, pdf1, video1, video2, video3, video4, video4, video5, video6, video7, video8, video9, video10, video11, video12, video13, video14, video15, video16, video17, video18, video19, video20, video21, video22, video23, video24, video25, video26, video27, video28, video29, video30 ]

# Github repo crawler
# Fetch GitHub Repositories using GitHub API
def fetch_github_repos(username):
    url = f"https://api.github.com/users/{username}/repos"
    response = requests.get(url)

    if response.status_code == 200:
        repos = response.json()
        repo_urls = [repo['html_url'] for repo in repos]
        return repo_urls
    else:
        print("Failed to fetch GitHub repositories")
        return []

# Add GitHub repositories to the list of loaders
github_username = "TheArchitect2000"
repo_urls = fetch_github_repos(github_username)

# Add GitHub repositories to loaders dynamically
for repo_url in repo_urls:
    loaders.append(WebBaseLoader(repo_url))

docs = []

for loader in loaders:
    try:
        loaded_docs = loader.load()
        for doc in loaded_docs:
            if isinstance(loader, YoutubeLoader):
                doc.metadata['type'] = 'YouTube'
            elif isinstance(loader, PyPDFLoader):
                doc.metadata['type'] = 'PDF'
            elif isinstance(loader, WebBaseLoader):
                doc.metadata['type'] = 'Web'
            else:
                doc.metadata['type'] = 'Other'
            print(doc.metadata)
            docs.append(doc)
    except Exception as e:
        print(f"Failed to load from {loader}: {e}")

✅ Connected to FidesInnova Blockchain!
Latest block: 3277666
{'source': 'https://www.fidesinnova.io/', 'title': 'FidesInnova-Verified Computations for Reliable Results - FidesInnova', 'language': 'en-US', 'type': 'Web'}
{'source': 'https://fidesinnova.io/devices/', 'title': 'Devices - FidesInnova', 'language': 'en-US', 'type': 'Web'}
{'source': 'https://fidesinnova.io/Contacts/', 'title': 'Contacts - FidesInnova', 'language': 'en-US', 'type': 'Web'}
{'source': 'https://fidesinnova.io/courses/', 'title': 'Courses Archive - FidesInnova', 'language': 'en-US', 'type': 'Web'}
{'source': 'https://linktr.ee/fidesinnova/', 'title': 'Fides Innova | Linktree', 'description': 'A decentralized verifiable computing platform', 'language': 'en', 'type': 'Web'}
{'source': 'https://explorer.fidesinnova.io/', 'title': 'Fides Innova Explorer', 'description': 'Explore the power of FidesInnova’s blockchain platform, designed to ensure secure, transparent, and verifiable computations for IoT devices. With z

In [4]:
############################# split the documents
splitter1 = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
totaldoc2 = splitter1.split_documents(docs)

############################# Creating Vector database
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

#######################
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Use the API key
# print(f"Using API Key: {api_key1}")
#######################

embedding1 = OpenAIEmbeddings(model="text-embedding-3-large") # api_key=api_key1 is removed as it's been already set in  os.environ["OPENAI_API_KEY"]

# on RAM memory
#mydb1 = Chroma.from_documents(totaldoc2, embedding=embedding1) 
# on storage
mydb1 = Chroma(
    collection_name="example_collection",
    embedding_function=embedding1,
    persist_directory="chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

mydb1.add_documents(totaldoc2)

/var/folders/dx/hkx51pqs1vxc2zsq74nxsjp80000gn/T/ipykernel_73500/2409493330.py:28: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  mydb1 = Chroma(


['30f1e5d7-8017-4f92-9901-f882eb76c64a',
 'eab21e10-10f6-453c-b84c-3a671a54702a',
 'f77190a7-b077-4fbd-a20a-0c6d7b35ea6d',
 '2363057c-5a86-4319-ae0e-3365c07ad6b0',
 '70bfd8bc-74f8-4ffb-ab90-ca2c2182e4c1',
 'e0912142-82b2-430c-ad3c-3b9c247dfd91',
 'cfff4e44-f827-4fb5-aab9-b0cf857dad79',
 'a622430d-5c60-416a-89bb-fb069c67e395',
 'cd292213-f1f4-476b-b1e5-2d66a85e480f',
 '3a095f2a-ead9-4d6d-adae-a5c342adfacc',
 'adf10348-9c0a-4a45-a1d4-fc13963a64c9',
 'bd19d9b2-5123-49de-97b8-5e9b6f8056c6',
 'b6187fba-6d8e-4939-be4d-412e94a2dde5',
 'beb2c7bc-f0e3-41e0-ae36-bb1ee9f0aff1',
 '3c884330-f5d4-4200-8e2f-3684b2b031d6',
 'a1e20992-aa80-4a8b-8454-c9e09e360d89',
 '2fcd8bd9-c4dd-4ffc-8ec0-f069c8f92b1a',
 '7bdd73c0-a70c-42ce-9560-ecf7b1a75f64',
 '46129544-11cc-404c-bfa3-440ffa69340c',
 '7e6fcd76-8723-4efa-9048-663ba76e4354',
 '1af57457-4553-43de-94a4-e38665d2f0ca',
 'cc75a301-dad2-4a8a-8745-2c761d30275a',
 '9a2e284a-b6e0-42e5-b495-f8d39f5651b0',
 'c4f1c1ca-a035-4549-b7f1-610beebc04eb',
 '1f74854b-6cca-

In [ ]:
# with open("a.txt","w") as f1:
#     f1.write(web1.load()[0].page_content)